In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

2024-02-24 12:21:14.755049: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-24 12:21:14.755204: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-24 12:21:14.978716: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
data=pd.read_csv('/kaggle/input/binary-classification-of-bank-churn-synthetic-data/Dataset.csv')

In [3]:
data.Exited.value_counts()

Exited
0    138071
1     36957
Name: count, dtype: int64

In [4]:
data

,Surname,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,...,France,Germany,Spain,Female,Male,Mem__no__Products,Cred_Bal_Sal,Bal_sal,Tenure_Age,Age_Tenure_product
0,2023,668.0,33.0,3.0,0.00,2.0,1,0,181449.97,0,...,1,0,0,0,1,0.0,0.000000,0.000000,0.090909,99.0
1,2024,627.0,33.0,1.0,0.00,2.0,1,1,49503.50,0,...,1,0,0,0,1,2.0,0.000000,0.000000,0.030303,33.0
2,1236,678.0,40.0,10.0,0.00,2.0,1,0,184866.69,0,...,1,0,0,0,1,0.0,0.000000,0.000000,0.250000,400.0
3,1362,581.0,34.0,2.0,148882.54,1.0,1,1,84560.88,0,...,1,0,0,0,1,1.0,1022.940581,1.760655,0.058824,68.0
4,491,716.0,33.0,5.0,0.00,2.0,1,1,15068.83,0,...,0,0,1,0,1,2.0,0.000000,0.000000,0.151515,165.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175023,2226,644.0,28.0,7.0,155060.41,1.0,1,0,29179.52,0,...,1,0,0,0,1,0.0,3422.225727,5.314015,0.250000,196.0
175024,2846,800.0,29.0,2.0,0.00,2.0,0,0,167773.55,0,...,1,0,0,1,0,0.0,0.000000,0.000000,0.068966,58.0
175025,1999,771.0,39.0,5.0,0.00,2.0,1,0,96270.64,0,...,1,0,0,0,1,0.0,0.000000,0.000000,0.128205,195.0
175026,1336,516.0,35.0,10.0,57369.61,1.0,1,1,101699.77,0,...,1,0,0,0,1,1.0,291.079505,0.564108,0.285714,350.0


In [5]:
target=data.Exited

In [6]:
xdata=data.drop('Exited',axis=1)

In [7]:
value=np.sum(target)
count=0
index=[]
for i in range(len(target)):
    if target[i]==0 and count<value:
        count+=1
        index.append(i)
    elif count>value:
        break

In [8]:
index.extend(list(np.where(target == 1)[0]))

In [9]:
np.random.shuffle(index)

In [10]:
target=target[index]
xdata=xdata.iloc[index]

In [11]:
from sklearn import preprocessing

In [12]:
xdata=preprocessing.scale(xdata)

In [13]:
validation_size=int(xdata.shape[0]*0.1)
test_size=int(xdata.shape[0]*0.1)
train_size=int(xdata.shape[0]*0.8)

In [14]:
validation_size

7391

In [15]:
train_data=xdata[:train_size]
validation_data=xdata[train_size:train_size+validation_size]
test_data=xdata[train_size+validation_size:]
train_target=target[:train_size]
validation_target=target[train_size:train_size+validation_size]
test_target=target[train_size+validation_size:]

In [16]:
input_size=24
output_size=2
hidden_layer_size=150
model=tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size,activation='tanh'),
    tf.keras.layers.Dense(hidden_layer_size,activation='tanh'),
    tf.keras.layers.Dense(hidden_layer_size,activation='tanh'),
    tf.keras.layers.Dense(output_size,activation='softmax')
])
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
call=tf.keras.callbacks.EarlyStopping(patience=2)
model.fit(train_data,train_target,batch_size=100,validation_data=(validation_data,validation_target),epochs=100,verbose=2,callbacks=[call])

Epoch 1/100
592/592 - 4s - loss: 0.4603 - accuracy: 0.7827 - val_loss: 0.4297 - val_accuracy: 0.8063 - 4s/epoch - 6ms/step
Epoch 2/100
592/592 - 2s - loss: 0.4322 - accuracy: 0.7985 - val_loss: 0.4289 - val_accuracy: 0.8045 - 2s/epoch - 4ms/step
Epoch 3/100
592/592 - 2s - loss: 0.4295 - accuracy: 0.7992 - val_loss: 0.4247 - val_accuracy: 0.8081 - 2s/epoch - 4ms/step
Epoch 4/100
592/592 - 2s - loss: 0.4269 - accuracy: 0.8008 - val_loss: 0.4291 - val_accuracy: 0.8026 - 2s/epoch - 4ms/step
Epoch 5/100
592/592 - 2s - loss: 0.4251 - accuracy: 0.8008 - val_loss: 0.4224 - val_accuracy: 0.8065 - 2s/epoch - 4ms/step
Epoch 6/100
592/592 - 2s - loss: 0.4239 - accuracy: 0.8027 - val_loss: 0.4233 - val_accuracy: 0.8054 - 2s/epoch - 4ms/step
Epoch 7/100
592/592 - 2s - loss: 0.4225 - accuracy: 0.8030 - val_loss: 0.4264 - val_accuracy: 0.7998 - 2s/epoch - 4ms/step


In [17]:
loss,accuracy=model.evaluate(test_data,test_target)

231/231 [==============================] - 1s 2ms/step - loss: 0.4313 - accuracy: 0.8015


In [18]:
print(f' Loss {loss:.2f} Accuracy : {accuracy*100:.2f}%')

 Loss 0.43 Accuracy : 80.15%
